#                             LSTM Language Model 
 
 Let's create a language model to generate Homer's Iliad, the same manner in which Andrej Karpathy created a Paul Graham generator in his [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). To do so, we'll use [Keras](https://keras.io/) to build a single-layer LSTM (Long-Short-Term Memory) model with 128 hidden nodes. 
 
 The model will take an arbituary length of characters (60 in this case) from Iliad to learn to predict the 61st character. It will have no knowledge of any word used in Iliad, only characters. By chaining the predictions together, we can generate new Iliad text.


First we create a function to load the source text, which is downloaded from [Gutenberg](http://www.gutenberg.org/ebooks/author/705). The source is cleaned by removing the prefaces, chapter introductions, footnotes, annotations, etc, anything that is not written by Homer himself.

In [32]:
def source_text(source):
    try:
        with open(source) as f:
            text = f.read().lower()   
        return text
    except OSError as e:
        print("Could not process file because of {}".format(str(e)))

We open the source text and print a few lines to visually inspect the text. 

In [33]:
file = "./Iliad.txt"
text = source_text(file)   
print('Length of source text:', len(text), 'sentences\n')
print('Source text:', '\n',text[0:200])

Length of source text: 880635 sentences

Source text: 
   achilles' wrath, to greece the direful spring
  of woes unnumber'd, heavenly goddess, sing!
  that wrath which hurl'd to pluto's gloomy reign
  the souls of mighty chiefs untimely slain;
  whose lim



 
 
 
It appears that we still need to get rid of the space (actually two spaces) in front of every new line. So let's shift every sentence to the left by 2 characters.

In [8]:
while "  " in text:
    text = text.replace("  ", "")
print(text[0:300])

achilles' wrath, to greece the direful spring
of woes unnumber'd, heavenly goddess, sing!
that wrath which hurl'd to pluto's gloomy reign
the souls of mighty chiefs untimely slain;
whose limbs unburied on the naked shore,
devouring dogs and hungry vultures tore.
since great achilles and atrides stro


The create_input_output() function creates input sentence of arbituary length (in this case 60 characters) and pairs it with an output character (the 61st character) to be the corresponding output label.

In [9]:
def create_input_output(text, length = 60, stepsize = 3 ):    
    predicting_sentences = []
    predicted_char = []    
    for char in range(0, len(text)-length, stepsize):
        predicting_sentences.append(text[char:char+length])
        predicted_char.append(text[char+length])        
    return predicting_sentences, predicted_char 

For example the 5th input sentence "to greece the direful spring of woes unnumber'd, heavenly" has a corresponding output character "g".

In [46]:
predicting_sentences, predicted_char = create_input_output(text)
print("Input sentence:", predicting_sentences[5], "\n")
print("Output label:", predicted_char[5])

Input sentence: , to greece the direful spring
of woes unnumber'd, heavenly  

Output label: g


Two dictionaries are created, of which one of them, char_to_index(), will map each character found in the source text to an index, while the other, index_to_char(), will map an index to a character. There are a total of 31 different characters.

In [10]:
def char_to_index(text):
    char2index = {char:i for i, char in enumerate(sorted(set(text)))}
    return char2index 

def index_to_char(text):
    index2char = {i:char for i, char in enumerate(sorted(set(text)))}
    return index2char

In [11]:
print("char_to_index:", char_to_index(text))

char_to_index: {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38, 'ć': 39}


For characters or words to be fed intp a deep learning model, the input sentences and output labels must be converted to tensors first. The function vectorize() will convert both the sentences and labels into one-hot vectors through the function vectorize().

In [12]:
def vectorize(sentences, char_to_index,predicted_char,length = 60): 
    
    input_sentence = np.zeros((len(sentences), length, len(char_to_index)))
    output_char = np.zeros((len(sentences), len(char_to_index)))
    
    for i, sentence in enumerate(sentences):
        for j, char in enumerate(sentence):
            input_sentence[i, j, char_to_index[char]] = 1
            
        output_char[i, char_to_index[predicted_char[i]]] = 1
        
    return input_sentence, output_char        

In [13]:
import numpy as np
predicting_sentences, predicted_char = create_input_output(text)
char2index = char_to_index(text)
index2char = index_to_char(text)
input_sentence, output_char = vectorize(predicting_sentences, char2index, predicted_char, length = 60)

The 1st sentence and its corresponding output label are shown below. 

In [28]:
print("One-hot input sentence of shape{}:".format(input_sentence.shape),"\n", input_sentence[0], "\n")
print("One-hot output label of shape{}:".format(output_char.shape),"\n", output_char[0])

One-hot input sentence of shape(280894, 60, 40): 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 

One-hot output label of shape(280894, 40): 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


A single-layer LSTM model with 128 hidden nodes is built using Keras. It is connected to a Dense layer which outputs a vector of 39 probabilities.

In [36]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(input_sentence.shape[1], input_sentence.shape[2]))) 
model.add(keras.layers.Dense(input_sentence.shape[2], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="rmsprop")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               86528     
_________________________________________________________________
dense_4 (Dense)              (None, 40)                5160      
Total params: 91,688
Trainable params: 91,688
Non-trainable params: 0
_________________________________________________________________


The model is trained for 60 epochs, which saw its cross-entropy loss declined from 2.2934 to 1.1536. Thereafter the entire model architecture and its weights are saved using model.save()

In [ ]:
model.fit(input_sentence, output_char, batch_size=128, epochs=60)
model.save("iliad_lstm_128_1layer_trained-at-60epoch.h5")

Epoch 1/60 
 
 
280894/280894 [==============================] - 155s 551us/step - loss: 2.2934 
 
Epoch 59/60 
 
280894/280894 [==============================] - 151s 538us/step - loss: 1.1574 


Epoch 60/60 
 
280894/280894 [==============================] - 150s 534us/step - loss: 1.1536

The model can be reloaded again anytime now that it has been saved by calling load_model() and compiling the model again.

In [ ]:
model = keras.models.load_model("./iliad lstm 128 60th epoch/iliad_lstm_128_1layer_trained-at-60epoch.h5")
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

The following function renormalization() can be used to renomalize the predicted probabilities. The higher the temperature, the more creative the generated text will be.

In [58]:
def renormalization(old_probabilities, temperature):    
    prob = np.log(old_probabilities)/temperature  
    new_probabilities = np.exp(prob)   
    return new_probabilities / np.sum(new_probabilities)

Using the renormalized probabilities, the function sample() will return the prediction, which is the character, out of a total of 40 characters, with the highest probability.

In [59]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = renormalization(preds, temperature) 
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Next we generate a seed sentence of length 60, and use this seed to generate new Iliad text of 500 characters.

In [ ]:
import random
import sys

start_index = random.randint(0, len(text) - input_sentence.shape[1] - 1)
generated_text = text[start_index: start_index + input_sentence.shape[1]]
print('Generating with seed: "' + generated_text + '"' + "\n")

print(generated_text, end = "")

for i in range(500):
    sampled = np.zeros((1, input_sentence.shape[1], input_sentence.shape[2]))
    
    for t, char in enumerate(generated_text):
        sampled[0, t, char2index[char]] = 1.
        
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature=0.5)
    next_char = index2char[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)

**Example at temperature 1.0:**

Generating with seed: "ean, owns my reign.  
  and his hush'd waves lie silent on the"  
  
  
  
ean, owns my reign.  
  and his hush'd waves lie silent on the glore,  
and art: he fronm'd a heaving brave depier,  
our all her anger natred band to greet.  
with thy distribis with the mournful sway  
far ounded in earth a the dispants undies.  
as still'd atoltm him with re dass'd."    

moont, and some arior of my manly of the plains,  
and pale our gloomy vengeances shall we stand,  
through all before their chalarj blow the way.  
with hersed fame the hunget in alreaty alm,  
demand the triphy and a groy he fall.  
but sopelive whom in ample gims ipplaced  
the fields the wo  

**Example at temperature 1.0:**

Generating with seed: "o shall thy pity and forbearance give  
  a weak old man to se"  

o shall thy pity and forbearance give  
  a weak old man to searcy morena; day.  
what yet our close but fatient seen of heaven,  
thus savight heavens, and glowine when rage rend,  
to twose the skies gifts, and heaven to will;  
and morcay so ?ige, leave fate and maid:  
mournful, atsin the trojans mourn'd bendate;  
arm from silent our rage and sacred keep."    

tid heaven recrioms but it not that hail!  
beholl; my emphy heeds, me, listence mend,  
sumpuse achilles they vingim; appear,  
nor eye his flamious inoved and round,  
let not and troy not persuas for his sire,  
ampl  

**Example at temperature 1.0**

Generating with seed: "e this menace, this insulting strain?  
  enormous boaster! do"    

e this menace, this insulting strain?  
  enormous boaster! doyrned had epach;  
their some desagg hear, from his day we spear:  
the vigour ob, the tanks of nep herce weigs  
confess'd the chief the flying braves of are,  
and thus he moves his daysing brother shared.  
that to he spoke, and send" the king of night,  
in yir the fields in arm'd his body train,  
gorove arms, whoms troy! forbids the vangat of bown.  
s"o favour townom reens, the assiust of he shall,  
and nation's vengeand, or his shule defear.  
dither'd the roughting, phoebus in edert  
and view his phrygian   

**Less creative sentence at temperature of 0.5, but higher fidelity:**

Generating with seed: "inguish'd o'er the field  
  by the broad glittering of the se"    

inguish'd o'er the field  
  by the broad glittering of the seats of war.  
but seat the last the field resounding shore,  
the whom the victim the deep they bower:  
the son of the mannated the deed of lence.  
so such of men the fame of man a spire,  
the shore the thunder long strong shorted plain.  
so the shall pierced his fate the power of heaven,  
their sheathe the heaven their stern deffarior crown'd,  
and stand defengeant on the short of fate.  
now with a seer shall same the lord of sight,  
the darkness from his arm shall tream of heaven.  
whom might they falt, an  

**Another at temperature of 0.5:**

Generating with seed: "his tent the chief retains,  
  safe to transport him to his n"    

his tent the chief retains,  
  safe to transport him to his nigh bore.  
so braves forbeas the trojan force in vain,  
with joy some son of jove a sons of fame:  
nor to his arms, and lear the trojan fields,  
to sumble steel, and glorious of the diren:  
the phoans of the darents of the skies,  
but the wall one imparion ranks the shore,  
the victor passing to the furion son:  
so springly war, and breathes in the fought,  
and the rise start the short of closed of the die.    

so said, and short, the fields of man contends;  
and all the grecian thrones spreads the grecians   

Reference:  
  
1) Deep Learning with Python - Francois Chollet   
2) Deep Learning with Keras - Antonio Gulli, Sujit Pal   
3)  [The Unreasonable Effectiveness of Recurrent Neural Networks - Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
        
        